In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep
from datetime import date
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append("../")
import src.herramientas_scrapeo as hs
import src.biblioteca as bb
import src.soporte as sp

In [ ]:
# Configuro las opciones de Selenium
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])#para ocultarme como robot
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('--incognito')#incognito window

In [2]:
# Declaro unas variables para usarlo después
productos_mercadona = bb.productos_mercadona_estructura.copy() # Estructura en formato diccionario del dataframe que voy a usar
recorrer = bb.listacategorias_merc.copy() # Una lista con todas las categorías del mercadona por las que liego quiero scrapear
errores = set() # Un set en el que se almacenarán las categorías que no han podido ser scrapeadas al commpleto, al ser un set no se repiten los objetos
today = date.today() # Creo una variable con la fecha actual

En esta pequeña casilla modifico las variables anteriores, para scrapear de nuevo las categorías que no han podido completarse
es decir, es solo para la segunda vez que recorremos

In [3]:

dfincompleto = pd.read_csv("../data/merc2023-01-20_suma.csv") # Cargo el csv con los datos incompletos
for cat_id in dfincompleto["category_id"].unique(): # Busco las categorías que he scrapeado al completo
    if cat_id in recorrer:
        recorrer.remove(cat_id) # Y las elimino de las lista a recorrer
    else:
        pass

# Scrapeamos mediante una función

In [ ]:
hs.scrap_mercadona(recorrer, productos_mercadona, errores)

# Pasamos a dataframe

In [ ]:
# Con los datos en formato diccionario, lo pasamos a dataframe
dfm = pd.DataFrame(productos_mercadona)


In [ ]:
# Crreo una copia por si acaso
dfm2 = dfm.copy()

In [ ]:
dfm2.head(2)

In [ ]:
# Checkeo el número de items que se han scrapeado de cada categoría
dfmcate_id = dfm2["category_id"].value_counts().reset_index()

In [ ]:
# Al ver que unas pocas categorías no tienen suficientes objetos scrapeados, las elimino y las añado de nuevo a la lista de errores
dfmcate_id.apply(lambda x: sp.revisar_errores(x["index"], x["category_id"], errores), axis = 1)

In [ ]:
# Eliminos todos los productos de las categorías que no se han scrapeado al completo, para poder scrapear luego todos los productos
for error in errores:
    dfm2 = dfm2[dfm2["category_id"] != error]

In [ ]:
# Elimino los posibles productos que no me sirven para mi análisis
dfm2.dropna(inplace=True)

# Almacenamos los datos

Si es la segunda vez, para almacenar todo en el mismo csv cargamos el csv anterior y lo cargamos, además de juntarlo con lo que hemos conseguido
y ya después lo guardamos

In [ ]:

dfsuma = pd.read_csv("../data/merc2023-01-20_suma.csv")
dfsuma2 = pd.concat([dfsuma, dfm2], axis = 0, join = "outer", ignore_index = True)
dfsuma2.to_csv(f"../data/merc{today}_suma.csv")

Si es la primera vez lo que hacemos es guardar estos datos como el CSV inicial en el que vamos acumulando datos

In [ ]:
dfm2.to_csv(f"../data/merc{today}_suma.csv")